# LLM tooling for D3.js

<p style="font-style: italic; font-size: 18px; color: dimgray;">ChatGPT and Gemini</p>

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
September 2025

-----

## Introduction

This notebook shows how to do parallel [Function Calling](https://ai.google.dev/gemini-api/docs/function-calling) workflows with Large Language Models (LLMs) of Gemini. 

The Raku package ["WWW::Gemini"](https://github.com/antononcube/Raku-WWW-Gemini), [AAp2], is used.

-----

## Setup

Load packages:

In [ ]:
use JSON::Fast;
use LLM::Functions;
use LLM::Tooling;
use Chemistry::Stoichiometry;


Define LLM access configurations:

In [ ]:
sink my $conf5-mini = llm-configuration('ChatGPT', model => 'gpt-5-mini');
sink my $conf41-mini = llm-configuration('ChatGPT', model => 'gpt-4.1-mini', :8192max-tokens, temperature => 0.4);
sink my $conf-gemini-flash = llm-configuration('Gemini', model => 'gemini-2.0-flash', :8192max-tokens, temperature => 0.4);

### JavaScript::D3

In [ ]:
#%javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [ ]:
#%js
js-d3-list-line-plot(10.rand xx 30, background => 'none')

----

## Diagram

Here is a [Mermaid-JS](https://mermaid.js.org) diagram that shows _single-pass_ LLM-and-tool interaction:


```mermaid
sequenceDiagram
    participant Developer
    participant Model as LLM Service (Gemini)
    participant Tool as External Function

    Note over Developer, Model: 1. Tool Definitions + Messages
    Developer->>Model: molecular-mass(molecule) <br> "What's the mass of SO2?"

    Note over Model: 2. Tool Calls
    Model->>Tool: molecular-mass("SO2")

    Note over Tool: 3. Execute Function Code
    Tool->>Model: {"mass": 64.058}

    Note over Model: 4. Results
    Model->>Developer: {"mass": 64.058}

    Note over Developer: 5. Final Response
    Developer->>Model: The molecular mass of SO2 is 64.058.
```


**Remark:** Instead of one invocation of a local tool -- as shown in the diagram -- often enough multiple LLM-&-local-computer interactions are used for LLM-tool computation.


----

## JavaScript graphics via [D3.js](https://d3js.org)

Define tools:

In [ ]:
sink my @js-tool-specs = (
{
    :name("js-d3-random-mondrian"),
    :description("Make the JavaScript code of a random Mondrian plot"),
    :parameters({
        :type("object"),
        :properties( {
            "\$width" => { :description("Width of the plot"), :type("integer") },
            "\$height" => { :description("Height of the plot"), :type("integer") }
            }),
        :required([]),
    }),
},
);

In [ ]:
sink my @js-tool-objects =
        LLM::Tool.new(&js-d3-random-mondrian),
        LLM::Tool.new(&js-d3-random-mandala)
        ;

sink my $conf = llm-configuration($conf-gemini-flash, tools => @js-tool-objects);
#sink my $conf = llm-configuration($conf41-mini, tools => @js-tool-objects);

In [ ]:
#% js
my $request = q:to/END/;
Generate a random Mondrian with dark gray background and image width == 450. 
Stop generating immediately when you obtain JavaScript / D3.js code.
Make sure the argument $color-palette to be Whatever.
END

my $res = 
    llm-synthesize([
        $request, 
        llm-prompt('NothingElse')('D3.js code')
        ], 
        e => llm-configuration($conf41-mini, tools => @js-tool-objects)
    ):echo;

$res .= subst(/ '```javascript' | '```'/, :g)

-----

## References

### Articles, blog posts

[AA1] Anton Antonov,
["LLM function calling workflows (Part 1, OpenAI)"](https://rakuforprediction.wordpress.com/2025/06/01/llm-function-calling-workflows-part-1-openai/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[AA2] Anton Antonov,
["LLM function calling workflows (Part 2, Google's Gemini)"](https://rakuforprediction.wordpress.com/2025/06/01/llm-function-calling-workflows-part-2-google-gemini/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[AA3] Anton Antonov,
["LLM function calling workflows (Part 3, Facilitation)"](https://rakuforprediction.wordpress.com/2025/06/08/llm-function-calling-workflows-part-3-facilitation/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[Gem1] Google Gemini,
["Gemini Developer API"](https://ai.google.dev/gemini-api/docs).

[WRI1] Wolfram Research, Inc.
["LLM-Related Functionality" guide](https://reference.wolfram.com/language/guide/LLMFunctions.html).



### Packages 

[AAp1] Anton Antonov,
[WWW::OpenAI Raku package](https://github.com/antononcube/Raku-WWW-OpenAI),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[WWW::Gemini Raku package](https://github.com/antononcube/Raku-WWW-Gemini),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).